# 키 확인

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))

True

# 입력일 종가 크롤링

In [5]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import schedule
import time
def crawling_input_date(date):
    '''
        date 형식입력 => yyyymmdd (ex 25/12/1 => 20251201)
        장이 열리지 않은 날(주말, 공휴일 등), 당일은 데이터 없음
    '''
    
    load_dotenv()
    key=os.getenv('KEY')
    url = "https://data-dbg.krx.co.kr/svc/apis/idx/krx_dd_trd"
    params = {
        "AUTH_KEY": key,
        "basDd": date,   # 기준일자 (당일 데이터 없는지 확인, 17:44분 기준 데이터 없음을 확인)
        'IDX_NM' : 'KRX 300',
        "REQ_TYPE": "json"
    }

    response = requests.get(url, params=params) 
    data = response.json()

    # JSON 파싱
    if "OutBlock_1" in data and data["OutBlock_1"]:
        df = pd.json_normalize(data["OutBlock_1"])
        df = df[["BAS_DD", "IDX_NM", "CLSPRC_IDX", "ACC_TRDVOL"]]
        
        target_idx = ["KRX 건설", "KRX 자동차", "KRX 헬스케어"] 
        df = df[df["IDX_NM"].isin(target_idx)]
        df.columns = ["날짜", "종목명", "종가(백만원)", "거래량(천주)"]
        print(df)
    else:
        print("데이터 없음")

if __name__ == '__main__':
    crawling_input_date(input('날짜를 입력하세요 형식ex. 20250101 :'))

날짜를 입력하세요 형식ex. 20250101 :20251230
          날짜       종목명  종가(백만원)   거래량(천주)
9   20251230   KRX 자동차  2529.04   8162435
11  20251230  KRX 헬스케어  4865.56  19150071
16  20251230    KRX 건설   824.65  19564420


# 어제의 종가까지 크롤링

In [11]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
def crawling_to_yesterday():
    '''
        2025/09/30 부터 오늘날짜(실제론 어제자 종가)까지
    '''
    
    load_dotenv()
    key=os.getenv('KEY')

    url = "https://data-dbg.krx.co.kr/svc/apis/idx/krx_dd_trd"

    start_date = datetime(2025, 9, 30)   # 시작일
    end_date   = datetime.today()  # 종료일

    all_data = []
    not_in_data = []

    while start_date <= end_date:
        date_str = start_date.strftime("%Y%m%d")  # 'YYYYMMDD' 형식
        params = {
            "AUTH_KEY": key,
            "basDd": date_str,
            "REQ_TYPE": "json"
        }
        response = requests.get(url, params=params)
        data = response.json()

        if "OutBlock_1" in data and data["OutBlock_1"]:
            # 원하는 IDX_NM만 필터링
            for row in data["OutBlock_1"]:
                if row["IDX_NM"] in ["KRX 건설", "KRX 자동차", "KRX 헬스케어"]:
                    all_data.append(row)

        # 휴무일을 확인할 필요가 있는 경우 주석 해제하면 출력(단 결측치가 생김)
        #else:
            ## 휴무일, 장 중에는 데이터가 없음
            #print(f"{date_str} 데이터 없음")
            ## 날짜 저장 리스트
            #not_in_data.append(date_str)

        start_date += timedelta(days=1)

    # DataFrame으로 변환 후 필요한 열만 추출
    df = pd.DataFrame(all_data)[["BAS_DD", "IDX_NM", "CLSPRC_IDX", "ACC_TRDVOL"]]
    df.columns = ["날짜", "종목명", "종가(백만원)", "거래량(천주)"]

    # 증감치열 추가
    df.sort_values(by=['날짜','종목명'],ascending=[True,True], inplace=True)
    df.reset_index(drop=True, inplace=True)

    check_data = df.iloc[:,-2].copy()
    KRX_items = set(df.iloc[:,1])

    data_count = len(check_data)
    KRX_count = len(KRX_items) # 3개(건설, 자동차, 헬스케어)
    date_count = int(data_count/KRX_count) # KRX_count의 묶음 갯수

    up_down_list = []
    # 전체데이터에 날짜당 아이템 갯수를 나누어 묶음들을 구함
    for date_idx in range(date_count):
        # 묶음 내부의 값들의 증감치 추가
        # 첫 묶음은 비교대상이 없기에 '시작일'로 지정
        if date_idx == 0:
            for start_idx in range(KRX_count):
                up_down_list.append(0)
        else:
            # 현재 묶음과 이전 묶음의 각각의 아이템의 크기를 비교 
            for udw_idx in range(KRX_count):
                # 전일대비 증감치를 숫자로 표기, 소수점 둘째까지 반올림
                up_down_list.append(round(float(check_data[date_idx*3 + udw_idx])-
                                          float(check_data[(date_idx-1)*3 + udw_idx]),2))

    df['전일대비'] = up_down_list
    df.to_csv('data/주가지수크롤링.csv')
    print('작업완료, 데이터 csv로 저장')

if __name__ == '__main__':
    crawling_to_yesterday()

작업완료
